Imports

In [ ]:
import requests
import cv2
import math
import numpy as np
import torch
import json
import ast
import os
from PIL import Image
from io import BytesIO
from tqdm import tqdm
from sklearn.cluster import KMeans
from retinaface import RetinaFace
from scipy.stats import pearsonr, wasserstein_distance
from skimage.metrics import structural_similarity as ssim


# %matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import ssl

# Disable SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

Open json image data

In [ ]:
with open('image_data.json', 'r') as f:
    data = json.load(f)

MiDaS model

In [ ]:

#model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas_large = torch.hub.load("intel-isl/MiDaS", "DPT_Large")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas_large.to(device)
midas.eval()
midas_large.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")


if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Functions

In [ ]:

def gauss(sigma):
    N = sigma * 3 + 0.5 // 1
    rangee = np.linspace(-N, N, num=int(2*N+1), dtype=np.float64)
    kernel = 1/(math.sqrt(2 * math.pi) * sigma) * np.exp(-rangee**2/(2*sigma**2))
    return (kernel / np.sum(kernel))
    
def load_images(page):
    request = f'https://www.wikiart.org/en/popular-paintings?json=2&page={page}'
    response = requests.get(request)
    return response.json()['Paintings']

def load_images_2(page):
    request = f'https://www.wikiart.org/en/popular-paintings?json=1&page={page}'
    response = requests.get(request)
    return response.json()

def painting_detailss_legacy(id):
    request = f'http://www.wikiart.org/en/App/Painting/ImageJson/{id}'
    response = requests.get(request)
    return response.json()

def painting_details(id):
    request = f'https://www.wikiart.org/en/api/2/Painting?id={id}'
    response = requests.get(request)
    #check if response is valid
    if response.status_code == 200:
        return response.json()
    else:
        return None

def popular_artists(page):
    request = f'http://www.wikiart.org/en/Popular-Artists/alltime/{page}?json=1&PageSize=100'
    response = requests.get(request)
    return response.json()

def paintings_by_artist(artist_url):
    request = f'http://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist_url}&json=2'
    response = requests.get(request)
    return response.json()

def most_popular_paintings(pagination_token=None):
    pagination_string = f'?&paginationToken={pagination_token}' if pagination_token is not None else ''
    request = f'https://www.wikiart.org/en/api/2/MostViewedPaintings{pagination_string}'
    response = requests.get(request)
    json_data = response.json()
    try:
        data = json_data['data']
        pagination_token = json_data['paginationToken']
        has_more = json_data['hasMore']
    except:
        data = json_data
        pagination_token = None
        has_more = False
    return data, pagination_token, has_more


def dictionaries_by_group_id(id=1):
    ## ID:
    ## 1: Art Movement
    ## 2: Style
    ## 3: Genre
    ## 4: Technique
    ## 7: Painting School or Group
    ## 10: Nation
    ## 11: Field
    request = f'http://www.wikiart.org/en/App/wiki/DictionariesJson/{id}'
    response = requests.get(request)
    return response.json()

def artists_by_dictionary(dictionary, seo_name, page):
    request = f'http://www.wikiart.org/en/App/{dictionary}/{seo_name}/{page}?json=1&PageSize=100'
    print(request)
    response = requests.get(request)
    return response

def load_image(image):
    img =  Image.open(BytesIO(requests.get(image['image']).content))
    return img

def load_np_image(image):
    img =  np.array(Image.open(BytesIO(requests.get(image['image']).content)))
    return img

def load_np_image_2(image):
    url = image['image']
    remove = '!Large.jpg'
    url = url.replace(remove, '')
    img =  np.array(Image.open(BytesIO(requests.get(url).content)))
    return img


def predict_depth_resize(image):
    im_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    input_batch = transform(im_bgr).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=im_bgr.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

    output = prediction.cpu().numpy()
    return output

def predict_depth(image):
    im_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    input_batch = transform(im_bgr).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

    output = prediction.cpu().numpy()
    return output.squeeze()

def predict_depth_large(image):
    im_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    input_batch = transform(im_bgr).to(device)

    with torch.no_grad():
        prediction = midas_large(input_batch)

    output = prediction.cpu().numpy()
    return output.squeeze()

def plot_3d(image, depth_map):
    if len(image.shape) < 3:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    if image.shape != depth_map.shape:
        image = cv2.resize(image, (depth_map.shape[1], depth_map.shape[0]))
    x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(x, y, depth_map, facecolors=image/255.0, rstride=1, cstride=1, cmap=cm.viridis, linewidth=0, antialiased=False, shade=False)
    ax.view_init(azim=70, elev=70)
    plt.show()

# def plot_3d_scale(image, depth_map):
#     if len(image.shape) < 3:
#         image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#     if image.shape != depth_map.shape:
#         image = cv2.resize(image, (depth_map.shape[1], depth_map.shape[0]))
#     x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
#     plt.subplot(1, 2, 1)
#     fig = plt.figure()
#     ax = fig.gca(projection='3d')
#     ax.plot_surface(x, y, depth_map, facecolors=image/255.0, rstride=1, cstride=1, cmap=cm.viridis, linewidth=0, antialiased=False, shade=False)
#     ax.view_init(azim=70, elev=70)
#     plt.subplot(1, 2, 2)
#     fig = plt.figure()
#     ax = fig.gca(projection='3d')
#     depth_normalized = (depth_map - np.min(depth_map)) / (np.max(depth_map) - np.min(depth_map))
#     ax.plot_surface(x, y, depth_map, rstride=1, cstride=1, facecolors=cm.viridis(depth_normalized), linewidth=0, antialiased=False, shade=False)
#     ax.view_init(azim=70, elev=70)
#     plt.show()

def plot_3d_scale(image, depth_map):
    if len(image.shape) < 3:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    if image.shape != depth_map.shape:
        image = cv2.resize(image, (depth_map.shape[1], depth_map.shape[0]))

    x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))

    # Create a single figure
    fig = plt.figure(figsize=(12, 6))

    # First subplot
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    ax1.plot_surface(y, x, depth_map, facecolors=image/255.0, rstride=1, cstride=1, cmap=cm.viridis, linewidth=0, antialiased=False, shade=False)
    ax1.view_init(azim=30, elev=60)

    # Second subplot
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    depth_normalized = (depth_map - np.min(depth_map)) / (np.max(depth_map) - np.min(depth_map))
    ax2.plot_surface(y, x, depth_map, rstride=1, cstride=1, facecolors=cm.viridis(depth_normalized), linewidth=0, antialiased=False, shade=False)
    ax2.view_init(azim=30, elev=60)

    plt.show()


Download and save image data

In [ ]:
# USE NA ENKRAT *NE DELA*

def build_dataset_old(data_dict, progress_bar, pagination_token=None, paintings_count=0, has_more=True, painting_index=0):   
    while has_more:
        data, new_token, has_more = most_popular_paintings(pagination_token)
    
        if 'Exception' in data:
            return build_dataset_old(data_dict, progress_bar, pagination_token, paintings_count, True, 0)
        else:
            pagination_token = new_token

        for i in range(painting_index, len(data)):
            painting = data[i]
            details = painting_details(painting['id'])
            if 'Exception' in details:
                return build_dataset_old(data_dict, progress_bar, pagination_token, paintings_count, has_more, i)

            painting_id = details.pop('id')
            data_dict[painting_id] = details
            progress_bar.update(1)

        paintings_count += len(data)
        
        if paintings_count > progress_bar.total:
            return data_dict, paintings_count
        
    return data_dict, paintings_count

def build_dataset_save(progress_bar, pagination_token=None, painting_index=0):   
    has_more=True
    data_dict = {}
    while has_more:
        data, pagination_token, has_more = most_popular_paintings(pagination_token)
    
        if 'Exception' in data:
            return data_dict, pagination_token, 0

        for i in range(painting_index, len(data)):
            painting = data[i]
            details = painting_details(painting['id'])
            if 'Exception' in details:
                return data_dict, pagination_token, i + 1

            painting_id = details.pop('id')
            details.pop('description')
            data_dict[painting_id] = details
            progress_bar.update(1)
        
    return data_dict, pagination_token, 0

        

In [ ]:
# PO DELIH 


def build_dataset(progress_bar, pagination_token=None, painting_index=0):   
    has_more=True
    data_dict = {}
    while has_more:
        try:
            data, pagination_token, has_more = most_popular_paintings(pagination_token)
        
            if 'Exception' in data:
                print('API 1 paintings', data)
                return data_dict, pagination_token, 0
        
        except KeyboardInterrupt:
            return data_dict, pagination_token, 0

        for i in range(painting_index, len(data)):
            painting_index = 0
            painting = data[i]
            try:
                details = painting_details(painting['id'])
            except KeyboardInterrupt:
                return data_dict, pagination_token, i
            
            if details == None:
                print('Error with painting details', painting)
                return data_dict, pagination_token, i
            
            if 'Exception' in details:
                print('API 2 details', details)
                return data_dict, pagination_token, i

            painting_id = details.pop('id')
            details.pop('description')
            data_dict[painting_id] = details
            progress_bar.update(1)
        
    return data_dict, pagination_token, 0

start_token = 'bj9CuXHXal%2bknEh%2foAMmIuP81uP3rQJss%2fyoYdMOmyapgSx0TJrhCEoNF6VfvUeOm%2bmDw%2fjhd%2f46zsk3PTrRKCn0BZx7khhslEtYt%2bijrsQ%3d'
start_index = 38
dict_index = 1

progress_bar = tqdm()
data_dict, token, index  = build_dataset(progress_bar, start_token, start_index)
progress_bar.close()

print('Token: ', token)
print('Index: ', index)
print('Length: ', len(data_dict))
print('Next dictionary index: ', dict_index + 1)

with open(f'data_{dict_index}.json', 'w') as outfile:
    json.dump(data_dict, outfile)

In [ ]:
# MERGE DATA

with open('all_data.json', 'r') as json_file:
    all_data = json.load(json_file)

for i in range(1, 5):
    file_path = f'data_{i}.json'
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
        all_data.update(data)
        os.remove(file_path)

print(len(all_data))

with open('all_data.json', 'w') as outfile:
    json.dump(all_data, outfile)



Method with faces

In [ ]:
# TEST FACE DETECTORS

import dlib
import face_detection

def test_face_detectors(painting):
    # check if painting is a dictionary or a path to an image
    if isinstance(painting, dict):
        image = load_np_image(painting)
    else:
        image = painting

    if image.shape[0] > image.shape[1]:
        figsize=(8, 6)
        subplot_1 = 2
        subplot_2 = 3
    else:
        figsize=(6, 8)
        subplot_1 = 3
        subplot_2 = 2
    
    fig = plt.figure(figsize=figsize)

    # 1 opencv - haar cascade
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces_cv = face_cascade.detectMultiScale(gray, 1.1, 4)
    cv_image = image.copy()
    
    if len(faces_cv) > 0:
        for (x, y, w, h) in faces_cv:
            cv2.rectangle(cv_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.subplot(subplot_1, subplot_2, 1)
    plt.axis('off')    
    plt.imshow(cv_image)
    plt.title('Haar cascade - opencv')

    # 2 dlib - HOG
    detector = dlib.get_frontal_face_detector()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces_dlib = detector(gray, 1)
    dlib_image = image.copy()
    
    if len(faces_dlib) > 0:
        for face in faces_dlib:
            cv2.rectangle(dlib_image, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)
    plt.subplot(subplot_1, subplot_2, 2)
    plt.axis('off')
    plt.imshow(dlib_image)
    plt.title('HOG - dlib')        
    
    # 3 face_detection - DSFD
    if len(faces_dlib) > 0:
        detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)
        faces_fd = detector.detect(image)
    else:
        faces_fd = []
    fd_image = image.copy()

    if len(faces_fd) > 0:
        for face in faces_fd:
            x1, y1, x2, y2 = int(face[0]), int(face[1]), int(face[2]), int(face[3])
            cv2.rectangle(fd_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.subplot(subplot_1, subplot_2, 3)
    plt.axis('off')
    plt.imshow(fd_image)
    plt.title('DSFD - face_detection')
   
    #4 dnn caffe
    face_net = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt", "res10_300x300_ssd_iter_140000.caffemodel")
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    faces_caffe = face_net.forward()
    caffe_image = image.copy()
    h, w = caffe_image.shape[:2]
    if len(faces_caffe) > 0:
        for i in range(0, faces_caffe.shape[2]):
            confidence = faces_caffe[0, 0, i, 2]
            if confidence > 0.5:
                box = faces_caffe[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                cv2.rectangle(caffe_image, (startX, startY), (endX, endY), (0, 255, 0), 2)

    plt.subplot(subplot_1, subplot_2, 4)
    plt.axis('off')
    plt.imshow(caffe_image)
    plt.title('SSD - caffe')


    #5 dlib cnn
    cnn_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
    faces_dlib_cnn = cnn_face_detector(image, 1)
    dlib_cnn_image = image.copy()
    
    if len(faces_dlib_cnn) > 0:
        for face in faces_dlib_cnn:
            x, y, w, h = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom()
            cv2.rectangle(dlib_cnn_image, (x, y), (w, h), (0, 255, 0), 2)
    plt.subplot(subplot_1, subplot_2, 5)  
    plt.axis('off')  
    plt.imshow(dlib_cnn_image)
    plt.title('MMOD CNN - dlib')

    # 6 retinaface
    if image.shape[2] == 4:
        image = image[:, :, :3]
        
    faces_rf = RetinaFace.detect_faces(image)
    rf_image = image.copy()
    
    if faces_rf:
        for face in faces_rf:  
            if len(face) == 0:
                break
            x1, y1, x2, y2 = faces_rf[face]['facial_area'][0], faces_rf[face]['facial_area'][1], faces_rf[face]['facial_area'][2], faces_rf[face]['facial_area'][3]
            cv2.rectangle(rf_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.subplot(subplot_1, subplot_2, 6)
    plt.axis('off')
    plt.imshow(rf_image)
    plt.title('RetinaFace')

    
    
    plt.tight_layout() 
    plt.show()


In [ ]:
#Examples for different face detectors

with open('image_data.json', 'r') as json_file:
    data = json.load(json_file)

    print(data['5772700bedc2cb3880bc7d83'])
    test_face_detectors(data['5772700bedc2cb3880bc7d83'])
    print(data['577277e1edc2cb3880d61b82'])
    test_face_detectors(data['577277e1edc2cb3880d61b82'])
    #test_face_detectors(data['577286a0edc2cb388004d4d9'])
    #test_face_detectors(data['57727ab6edc2cb3880df967a'])


In [ ]:
# Functions for first method

def  fit_plane(x, y, z, plot=False):
    A = np.column_stack((x, y, np.ones_like(x)))
    coefficients, residuals, _, _ = np.linalg.lstsq(A, z, rcond=None)
    a, b, c = coefficients
    normal = np.array([a, b, -1]) / np.linalg.norm(np.array([a, b, -1]))
    normal_z0 = np.array([0, 0, 1])
    angle = np.arccos(np.abs(np.dot(normal, normal_z0)))
    if plot:
        xx, yy = np.meshgrid(np.arange(0, 1, 0.1), np.arange(0, 1, 0.1))
        fig = plt.figure()
        ax = fig.gca(projection='3d')
        ax.scatter(x, y, z, c='r', s=100)
        ax.plot_surface(xx, yy, a * xx + b * yy + c, alpha=0.5)
        ax.view_init(azim=70, elev=70)
        plt.show()

    
    return angle, np.degrees(angle), a, b, c

def fit_plane_with_faces(image, plot=False):
    f = 17 #mm
    sensor_height = 32 #mm
    face_height = 185 #mm
    faces_rf = RetinaFace.detect_faces(image)
    points_x = []
    points_y = []
    points_z = []
    
    copy = image.copy()

    for i, face in enumerate(faces_rf):  
        if len(face) == 0:
            break
        x1, y1, x2, y2 = faces_rf[face]['facial_area'][0], faces_rf[face]['facial_area'][1], faces_rf[face]['facial_area'][2], faces_rf[face]['facial_area'][3]
        face_height_px = y2 - y1
        image_height = image.shape[0]
        d = (f * face_height * image_height) / (face_height_px * sensor_height)
        points_x.append((x1+x2)/2)
        points_y.append((y1+y2)/2)
        points_z.append(d)
        
        if plot:
            copy = cv2.rectangle(copy, (x1, y1), (x2, y2), (0, 255 / len(faces_rf) * (i + 1), 0), 2)
    if len(points_x) == 0:
        return -1, -1
    
    if plot:
        plt.imshow(copy)
        plt.show()
    
    # normalize points 
    points_x = np.array(points_x) / image.shape[1]
    points_y = np.array(points_y) / image.shape[0]
    points_z = np.max(points_z) - np.array(points_z) # rotate coordinate system
    points_z = (points_z - np.mean(points_z)) / np.std(points_z) # normalize
    
    angle, angle_deg, a, b, c = fit_plane(points_x, points_y, points_z, plot)
    
    return angle, angle_deg, a, b, c

def compare_faces_wtih_depth(image, plot=False):
    f = 17 #mm
    sensor_height = 32 #mm
    face_height = 185 #mm
    faces_rf = RetinaFace.detect_faces(image)
    print(faces_rf)
    points_x = []
    points_y = []
    points_z = []
    
    copy = image.copy()

    for i, face in enumerate(faces_rf):  
        if len(face) == 0:
            break
        x1, y1, x2, y2 = faces_rf[face]['facial_area'][0], faces_rf[face]['facial_area'][1], faces_rf[face]['facial_area'][2], faces_rf[face]['facial_area'][3]
        face_height_px = y2 - y1
        image_height = image.shape[0]
        d = (f * face_height * image_height) / (face_height_px * sensor_height)
        points_x.append((x1+x2)/2)
        points_y.append((y1+y2)/2)
        points_z.append(d)
        
        if plot:
            copy = cv2.rectangle(copy, (x1, y1), (x2, y2), (0, 255 / len(faces_rf) * (i + 1), 0), 2)
    if len(points_x) == 0:
        return -1, -1
    if plot:
        plt.imshow(copy)
        plt.show()
    

    depth_map = predict_depth(image)
    x_factor = depth_map.shape[1] / image.shape[1]
    y_factor = depth_map.shape[0] / image.shape[0]
    depth_map_x = np.array(points_x) * x_factor
    depth_map_y = np.array(points_y) * y_factor
    depth_map_z = np.zeros_like(points_z)
    
    # normalize points 
    points_x = np.array(points_x) / image.shape[1]
    points_y = np.array(points_y) / image.shape[0]
    points_z = np.max(points_z) - np.array(points_z) # rotate coordinate system
    points_z = (points_z - np.mean(points_z)) / np.std(points_z) # normalize


    x, y = np.meshgrid(np.arange(depth_map.shape[1]), np.arange(depth_map.shape[0]))
    for i in range(len(points_z)):
        x_i = x[np.where(np.sqrt((x - depth_map_x[i])**2 + (y - depth_map_y[i])**2) < 10)] 
        y_i = y[np.where(np.sqrt((x - depth_map_x[i])**2 + (y - depth_map_y[i])**2) < 10)]
        dist = np.mean(depth_map[y_i, x_i])
        depth_map_z[i] = dist
   
   
    if (plot):
        fig = plt.figure()
        ax = fig.gca(projection='3d')
        ax.plot_surface(x, y, depth_map, cmap=cm.viridis, linewidth=0, antialiased=False, alpha=0.5)
        colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
        ax.scatter(depth_map_x, depth_map_y, depth_map_z, c=colors[:len(depth_map_x)], s=100)
        ax.view_init(azim=70, elev=70)
        plt.show()

    depth_map_z = (depth_map_z - np.mean(depth_map_z)) / np.std(depth_map_z) # normalize

    mse = np.sum(np.square(points_z - depth_map_z)) / len(points_z)
    rmse = np.sqrt(mse)

    return mse, rmse

# THIS IS THE FUNCTION THAT WAS USED TO CALCULATE THE DEPTH OF THE FACES IN THE PAINTINGS
def find_depth_faces(painting):
    f = 17 #mm
    sensor_height = 32 #mm
    face_height = 185 #mm
    retinaface = ast.literal_eval(painting['retinaface'])
    
    if len(retinaface) == 0:
        return painting
    
    points_x = []
    points_y = []
    points_z = []

    # image = load_np_image(painting)

    for face_id in retinaface:
        face = retinaface[face_id]
        landmarks = face['landmarks']
        eyes = np.add(landmarks['right_eye'], landmarks['left_eye']) / 2
        mouth = np.add(landmarks['mouth_left'], landmarks['mouth_right']) / 2
        dist = np.sqrt((eyes[0] - mouth[0])**2 + (eyes[1] - mouth[1])**2)
        face_height_px = dist * 3
        image_height = painting['height']
        d = (f * face_height * image_height) / (face_height_px * sensor_height)
        x1, y1, x2, y2 = face['facial_area'][0], face['facial_area'][1], face['facial_area'][2], face['facial_area'][3]
        points_x.append((x1 + x2) / 2)
        points_y.append((y1 + y2) /2)
        points_z.append(d)

        # face_height_box = y2 - y1
        # d2 = (f * face_height * image_height) / (face_height_box * sensor_height)
        ## draw points
        #diff_vector = eyes - mouth
        #image = cv2.circle(image, (int(eyes[0]), int(eyes[1])), 5, (0, 0, 255), -1)
        #image = cv2.circle(image, (int(mouth[0]), int(mouth[1])), 5, (255, 0, 0), -1)
        #image = cv2.circle(image, (int(eyes[0] + 1.5 * diff_vector[0]), int(eyes[1] + 1.5* diff_vector[1])), 5, (0, 255, 255), -1)
        #image = cv2.circle(image, (int(eyes[0] - 1.5 * diff_vector[0]), int(eyes[1] - 1.5 * diff_vector[1])), 5, (0, 255, 255), -1)

        ## draw bounding box
        #image = cv2.rectangle(image, (face['facial_area'][0], face['facial_area'][1]), (face['facial_area'][2], face['facial_area'][3]), (0, 255, 0), 2)

        ## write text
        #image = cv2.putText(image, str(round(d, 2)), (int(eyes[0]), int(eyes[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        #image = cv2.putText(image, str(round(d2, 2)), (int(eyes[0]), int(eyes[1]) + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    # plt.imshow(image)
    # plt.show()

    ## NORMALIZE

    points_x = np.array(points_x) / painting['width']
    points_y = np.array(points_y) / painting['height']
    points_z = np.max(points_z) - np.array(points_z) 
    points_z = (points_z - np.mean(points_z)) / np.std(points_z)
    angle, angle_deg, a, b, c = fit_plane(points_x, points_y, points_z)
    painting['face_plane'] = {
        'angle': angle,
        'angle_deg': angle_deg,
        'a': a,
        'b': b,
        'c': c
    }
    return painting


In [ ]:
# Fit plane with faces and save results

with open('all_data_new_2.json', 'r') as json_file:
    data = json.load(json_file)

for id_ in data:
    if 'face_plane' in data[id_]:
        continue
    data[id_] = find_depth_faces(data[id_])

with open('all_data_new_2.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [ ]:
# Clustering BY FACE PLANE

import pandas as pd

with open('image_data.json', 'r') as json_file:
    data = json.load(json_file)

df_paintings = {}
for id_ in data:
    retina_face = ast.literal_eval(data[id_]['retinaface'])
    if 'face_plane' in data[id_] and len(retina_face) >= 2:
        attributes = data[id_]['face_plane']
        attributes['title'] = data[id_]['title']
        attributes['artist'] = data[id_]['artistName']
        if 'completitionYear' in data[id_]:
            attributes['year'] = data[id_]['completitionYear']
        df_paintings[id_] = attributes

# BUILD A DATAFRAME

df = pd.DataFrame.from_dict(df_paintings, orient='index')

# replace NaN values 
df = df.fillna(-1)

# remove rows with NaN values
#df = df.dropna()

# only rows with NaN values
#df = df[df.isna().any(axis=1)]


df

Fit plane to face data

In [ ]:
# PLOT faces and plane

def plot_faces(painting):
    f = 17 #mm
    sensor_height = 32 #mm
    face_height = 185 #mm

    image = load_np_image(painting)
   
    retinaface = ast.literal_eval(painting['retinaface'])
    
    points_x = []
    points_y = []
    points_z = []

    # image = load_np_image(painting)

    for face_id in retinaface:
        face = retinaface[face_id]
        landmarks = face['landmarks']
        eyes = np.add(landmarks['right_eye'], landmarks['left_eye']) / 2
        mouth = np.add(landmarks['mouth_left'], landmarks['mouth_right']) / 2
        dist = np.sqrt((eyes[0] - mouth[0])**2 + (eyes[1] - mouth[1])**2)
        face_height_px = dist * 3
        image_height = painting['height']
        d = (f * face_height * image_height) / (face_height_px * sensor_height)
        x1, y1, x2, y2 = face['facial_area'][0], face['facial_area'][1], face['facial_area'][2], face['facial_area'][3]
        points_x.append((x1 + x2) / 2)
        points_y.append((y1 + y2) /2)
        points_z.append(d)
       
    ## NORMALIZE

    points_x = np.array(points_x) / painting['width']
    points_y = np.array(points_y) / painting['height']
    points_z = np.max(points_z) - np.array(points_z) 
    points_z = (points_z - np.mean(points_z)) / np.std(points_z)

    color_z = points_z + np.abs(np.min(points_z))
    color_z = color_z / np.max(color_z)
    
    colormap_name = 'cool'
    cmap = plt.cm.get_cmap(colormap_name)

    # draw rectangles
    copy = image.copy()
    for i, face_id in enumerate(retinaface):
        face = retinaface[face_id]
        x1, y1, x2, y2 = face['facial_area'][0], face['facial_area'][1], face['facial_area'][2], face['facial_area'][3]
        
        color = cmap(color_z[i])
        copy = cv2.rectangle(copy, (x1, y1), (x2, y2), (color[0] *255, color[1] *255, color[2] *255), 2)
    
    plt.imshow(copy)
    plt.yticks([])
    plt.xticks([])
    plt.show()

    a,b,c = painting['face_plane']['a'], painting['face_plane']['b'], painting['face_plane']['c']
    x, y = x, y = np.meshgrid(np.arange(0, 1, 1/image.shape[1]), np.arange(0, 1, 1/ image.shape[0]))
    zr = a*x + b*y + c
    zr = zr * -1
    z0 = np.zeros_like(zr)

    z_bottom = np.where(z0 < zr, z0, np.nan)
    z_top = np.where(z0 > zr, z0, np.nan)

    size_z = points_z + np.abs(np.min(points_z))
    size_z = (np.max(size_z) - size_z + 1) * 20
   
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ## Plane
    ax.plot_surface(y, x, z_bottom, alpha=0.25, color='gray') 
    ax.plot_surface(y, x, zr, alpha=0.25, color='mediumspringgreen') 
    ax.plot_surface(y, x, z_top, alpha=0.25, color='gray') 
    ## Points
    plot = ax.scatter(points_y, points_x, points_z, c=points_z, alpha=1.0, cmap=colormap_name)
    ax.set_xlabel('Y')
    ax.set_ylabel('X')
    ax.set_zlabel('Z')
    ax.invert_zaxis()
    #ax.view_init(azim=0, elev=70)
    fig.colorbar(plot)
    plt.show()




def plot_results(image, points_x, points_y, points_z, a, b, c):
    x, y = np.meshgrid(np.arange(0, 1, 1/image.shape[1]), np.arange(0, 1, 1/ image.shape[0]))
    x = np.flip(x)
    zr = a*x + b*y + c
    z0 = np.zeros_like(zr)

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ## Plane
    ax.plot_surface(x, y, zr, alpha=2.0, color='o') 
    ## Points
    ax.scatter(points_x, points_y, points_z, c=points_z, alpha=0.1, cmap='Wistia')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    #ax.view_init(azim=70, elev=70)
    plt.show()


    
count = 0
for id_ in data:
    if count > 10:
        break
    painting = data[id_]
    retinaface = ast.literal_eval(painting['retinaface'])
    if len(retinaface) == 7:
            print(painting)
            print(painting['retinaface'])
            plot_faces(painting)
            count += 1



MiDaS

In [ ]:
# MiDaS - model comparison

def compare_midas_models(painting):
    print(painting['title'], painting['artistName'], painting['completitionYear'])

    image = load_np_image(painting)
    depth_map = predict_depth(image)
    depth_map_large = predict_depth_large(image)

    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.xticks([]) 
    plt.yticks([]) 
    plt.title('Original')
    plt.subplot(1, 3, 2)
    plt.imshow(depth_map)
    plt.xticks([])  
    plt.yticks([])  
    plt.title('MiDaS hybrid')
    plt.subplot(1, 3, 3)
    plt.imshow(depth_map_large)
    plt.xticks([])
    plt.yticks([])
    plt.title('MiDaS large')
    plt.show()

In [ ]:
# MiDaS - model comparison

import ipywidgets as widgets
from IPython.display import display

with open('image_data.json', 'r') as json_file:
    data = json.load(json_file)

# Create an Output widget
out = widgets.Output(layout={'border': '1px solid black', 'width': '500px', 'height': '500px', 'overflow': 'scroll'})

counter = 0
key_list = list(data.keys())
#with out:
for i in range(0, 100, 1):
    id_ = key_list[i]
    painting = data[id_]
    compare_midas_models(painting)

#display(out)

In [ ]:
# Shrink depth maps to 100x100 and convert to float32

with open('image_data.json', 'r') as json_file:
    data = json.load(json_file)

for id_ in data:
    depth_map = np.load(f'depth_maps/{id_}.npy')
    depth_map = (depth_map / 255.0).astype(np.float32)
    depth_map = cv2.resize(depth_map, (100, 100))
    np.save(f'depth_maps_100/{id_}.npy', depth_map)

In [ ]:
# Show 3d image and plot

with open('image_data.json', 'r') as json_file:
    data = json.load(json_file)

# Create an Output widget
#out = widgets.Output(layout={'border': '1px solid black', 'width': '500px', 'height': '500px', 'overflow': 'scroll'})

counter = 0
key_list = list(data.keys())
#with out:
for i in range(0, len(data), 1):
    id_ = key_list[i]
    painting = data[id_]

    if 'fallen angel' in painting['title'].lower():
        print(painting)
        image = load_np_image(painting)
        depth_map = predict_depth(image)

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.yticks([])
        plt.xticks([])
        plt.subplot(1, 2, 2)
        plt.imshow(depth_map)
        plt.yticks([])
        plt.xticks([])
        plt.show()

        #plot_3d(image, depth_map)
        plot_3d_scale(image, depth_map)
        
        break
    
    

Clustering and showcase


In [ ]:
# COMPARE IMAGES WITH DIFFERENT DEPTH MAPS

def plot_distances(id_):
    with open('image_data.json', 'r') as json_file:
        data = json.load(json_file)
    
    depth_map_1 = np.load(f'depth_maps_100/{id_}.npy')

    bar = tqdm(total=len(data))
    # EUCLEDIAN DISTANCE
    lowest_eucled = 99999999999999999
    lowest_eucled_id = None

    # MANHATTAN DISTANCE
    lowest_manhattan = 99999999999999999
    lowest_manhattan_id = None

    # CORRELATION COEFFICIENT
    correlation_coefficient = -1
    correlation_coefficient_id = None

    # STRUCTURAL SIMILARITY
    structural_similarity = -1
    structural_similarity_id = None

    # JACCARD SIMILARITY
    jaccard_similarity = 0
    jaccard_similarity_id = None

    # WASSERSTEIN DISTANCE - EMD
    wasserstein_dist = 99999999999999999
    wasserstein_dist_id = None
    
    # COSINE SIMILARITY
    cosine_similarity = -1
    cosine_similarity_id = None


    for id_2 in data:

        bar.update(1)
        if id_ == id_2:
            continue
        
        depth_map_2 = np.load(f'depth_maps_100/{id_2}.npy')
        
        # EUCLEDIAN DISTANCE
        eucled = np.sqrt(np.sum((depth_map_1 - depth_map_2)**2))
        if eucled < lowest_eucled:
            lowest_eucled = eucled
            lowest_eucled_id = id_2

        # MANHATTAN DISTANCE
        manhattan = np.sum(np.abs(depth_map_1 - depth_map_2))
        if manhattan < lowest_manhattan:
            lowest_manhattan = manhattan
            lowest_manhattan_id = id_2

        # CORRELATION COEFFICIENT
        cc = pearsonr(depth_map_1.flatten(), depth_map_2.flatten())[0]
        if cc > correlation_coefficient:
            correlation_coefficient = cc
            correlation_coefficient_id = id_2

        # STRUCTURAL SIMILARITY
        ss = ssim(depth_map_1, depth_map_2, data_range=255)
        if ss > structural_similarity:
            structural_similarity = ss
            structural_similarity_id = id_2

        # JACCARD SIMILARITY
        threshold_1 = np.mean(depth_map_1)
        threshold_2 = np.mean(depth_map_2)
        _, binary_1 = cv2.threshold(depth_map_1, threshold_1, 255, cv2.THRESH_BINARY)
        _, binary_2 = cv2.threshold(depth_map_2, threshold_2, 255, cv2.THRESH_BINARY)
        intersection = np.logical_and(binary_1, binary_2)
        union = np.logical_or(binary_1, binary_2)
        jaccard = intersection.sum() / union.sum()
        if jaccard > jaccard_similarity:
            jaccard_similarity = jaccard
            jaccard_similarity_id = id_2

        # WASSERSTEIN DISTANCE - EMD
        wasserstein = wasserstein_distance(depth_map_1.flatten(), depth_map_2.flatten())
        if wasserstein < wasserstein_dist:
            wasserstein_dist = wasserstein
            wasserstein_dist_id = id_2

        # COSINE SIMILARITY
        cosine = np.dot(depth_map_1.flatten(), depth_map_2.flatten()) / (np.linalg.norm(depth_map_1.flatten()) * np.linalg.norm(depth_map_2.flatten())) 
        if cosine > cosine_similarity:
            cosine_similarity = cosine
            cosine_similarity_id = id_2

    # plot results
    plt.figure(figsize=(5, 10))
    plt.subplot(4, 2, 1)
    plt.imshow(load_np_image(data[id_]))
    plt.title(data[id_]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 2)
    plt.imshow(np.load(f'depth_maps/{id_}.npy'))
    plt.title('Original')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 3)
    plt.imshow(load_np_image(data[lowest_eucled_id]))
    plt.title(data[lowest_eucled_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 4)
    plt.imshow(np.load(f'depth_maps/{lowest_eucled_id}.npy'))
    plt.title(f'eucled: {lowest_eucled:.2f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 5)
    plt.imshow(load_np_image(data[lowest_manhattan_id]))
    plt.title(data[lowest_manhattan_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 6)
    plt.imshow(np.load(f'depth_maps/{lowest_manhattan_id}.npy'))
    plt.title(f'manhattan: {lowest_manhattan:.2f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 7)
    plt.imshow(load_np_image(data[correlation_coefficient_id]))
    plt.title(data[correlation_coefficient_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 8)
    plt.imshow(np.load(f'depth_maps/{correlation_coefficient_id}.npy'))
    plt.title(f'r: {correlation_coefficient:.2f}')
    plt.yticks([])
    plt.xticks([])
    plt.show() 
    
    plt.figure(figsize=(5, 10))
    plt.subplot(4, 2, 1)
    plt.imshow(load_np_image(data[structural_similarity_id]))
    plt.title(data[structural_similarity_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 2)
    plt.imshow(np.load(f'depth_maps/{structural_similarity_id}.npy'))
    plt.title(f'SSIM: {structural_similarity:.2f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 3)
    plt.imshow(load_np_image(data[jaccard_similarity_id]))
    plt.title(data[jaccard_similarity_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 4)
    plt.imshow(np.load(f'depth_maps/{jaccard_similarity_id}.npy'))
    plt.title(f'J: {jaccard_similarity:.2f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 5)
    plt.imshow(load_np_image(data[wasserstein_dist_id]))
    plt.title(data[wasserstein_dist_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 6)
    plt.imshow(np.load(f'depth_maps/{wasserstein_dist_id}.npy'))
    plt.title(f'WD: {wasserstein_dist:.2f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 7)
    plt.imshow(load_np_image(data[cosine_similarity_id]))
    plt.title(data[cosine_similarity_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 8)
    plt.imshow(np.load(f'depth_maps/{cosine_similarity_id}.npy'))
    plt.title(f'CSIM: {cosine_similarity:.2f}')
    plt.yticks([])
    plt.xticks([])
    plt.show()      

    bar.close() 

In [ ]:

def plot_distances(id_):
    with open('image_data.json', 'r') as json_file:
        data = json.load(json_file)
    
    depth_map_1 = np.load(f'depth_maps_100/{id_}.npy')

    bar = tqdm(total=len(data))
    # EUCLEDIAN DISTANCE
    lowest_eucled = 99999999999999999
    lowest_eucled_id = None

    # MANHATTAN DISTANCE
    lowest_manhattan = 99999999999999999
    lowest_manhattan_id = None

    # CORRELATION COEFFICIENT
    correlation_coefficient = -1
    correlation_coefficient_id = None

    # STRUCTURAL SIMILARITY
    structural_similarity = -1
    structural_similarity_id = None

    # JACCARD SIMILARITY
    jaccard_similarity = 0
    jaccard_similarity_id = None

    # WASSERSTEIN DISTANCE - EMD
    wasserstein_dist = 99999999999999999
    wasserstein_dist_id = None
    
    # COSINE SIMILARITY
    cosine_similarity = -1
    cosine_similarity_id = None


    for id_2 in data:

        bar.update(1)
        if id_ == id_2:
            continue
        
        depth_map_2 = np.load(f'depth_maps_100/{id_2}.npy')
        
        # EUCLEDIAN DISTANCE
        eucled = np.sqrt(np.sum((depth_map_1 - depth_map_2)**2))
        if eucled < lowest_eucled:
            lowest_eucled = eucled
            lowest_eucled_id = id_2

        # MANHATTAN DISTANCE
        manhattan = np.sum(np.abs(depth_map_1 - depth_map_2))
        if manhattan < lowest_manhattan:
            lowest_manhattan = manhattan
            lowest_manhattan_id = id_2

        # CORRELATION COEFFICIENT
        cc = pearsonr(depth_map_1.flatten(), depth_map_2.flatten())[0]
        if cc > correlation_coefficient:
            correlation_coefficient = cc
            correlation_coefficient_id = id_2

        # STRUCTURAL SIMILARITY
        ss = ssim(depth_map_1, depth_map_2, data_range=1)
        if ss > structural_similarity:
            structural_similarity = ss
            structural_similarity_id = id_2

        # JACCARD SIMILARITY
        threshold_1 = np.mean(depth_map_1)
        threshold_2 = np.mean(depth_map_2)
        _, binary_1 = cv2.threshold(depth_map_1, threshold_1, 255, cv2.THRESH_BINARY)
        _, binary_2 = cv2.threshold(depth_map_2, threshold_2, 255, cv2.THRESH_BINARY)
        intersection = np.logical_and(binary_1, binary_2)
        union = np.logical_or(binary_1, binary_2)
        jaccard = intersection.sum() / union.sum()
        if jaccard > jaccard_similarity:
            jaccard_similarity = jaccard
            jaccard_similarity_id = id_2

        # WASSERSTEIN DISTANCE - EMD
        wasserstein = wasserstein_distance(depth_map_1.flatten(), depth_map_2.flatten())
        if wasserstein < wasserstein_dist:
            wasserstein_dist = wasserstein
            wasserstein_dist_id = id_2

        # COSINE SIMILARITY
        cosine = np.dot(depth_map_1.flatten(), depth_map_2.flatten()) / (np.linalg.norm(depth_map_1.flatten()) * np.linalg.norm(depth_map_2.flatten())) 
        if cosine > cosine_similarity:
            cosine_similarity = cosine
            cosine_similarity_id = id_2

    # plot results
    plt.figure(figsize=(5, 8))
    plt.subplot(4, 2, 1)
    plt.imshow(load_np_image(data[id_]))
    plt.title(data[id_]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 2)
    plt.imshow(np.load(f'depth_maps/{id_}.npy'))
    plt.title('Original')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 3)
    plt.imshow(load_np_image(data[lowest_eucled_id]))
    plt.title(data[lowest_eucled_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 4)
    plt.imshow(np.load(f'depth_maps/{lowest_eucled_id}.npy'))
    plt.title(f'Eucledian distance: {lowest_eucled:.3f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 5)
    plt.imshow(load_np_image(data[lowest_manhattan_id]))
    plt.title(data[lowest_manhattan_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 6)
    plt.imshow(np.load(f'depth_maps/{lowest_manhattan_id}.npy'))
    plt.title(f'Manhattan distance: {lowest_manhattan:.3f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 7)
    plt.imshow(load_np_image(data[correlation_coefficient_id]))
    plt.title(data[correlation_coefficient_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 8)
    plt.imshow(np.load(f'depth_maps/{correlation_coefficient_id}.npy'))
    plt.title(f'Correlation coef.: {correlation_coefficient:.3f}')
    plt.yticks([])
    plt.xticks([])
    plt.show() 
    
    plt.figure(figsize=(5, 8))
    plt.subplot(4, 2, 1)
    plt.imshow(load_np_image(data[structural_similarity_id]))
    plt.title(data[structural_similarity_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 2)
    plt.imshow(np.load(f'depth_maps/{structural_similarity_id}.npy'))
    plt.title(f'Structural similarity: {structural_similarity:.3f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 3)
    plt.imshow(load_np_image(data[jaccard_similarity_id]))
    plt.title(data[jaccard_similarity_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 4)
    plt.imshow(np.load(f'depth_maps/{jaccard_similarity_id}.npy'))
    plt.title(f'Jaccard Similarity: {jaccard_similarity:.3f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 5)
    plt.imshow(load_np_image(data[wasserstein_dist_id]))
    plt.title(data[wasserstein_dist_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 6)
    plt.imshow(np.load(f'depth_maps/{wasserstein_dist_id}.npy'))
    plt.title(f'Wasserstein distance: {wasserstein_dist:.3f}')
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 7)
    plt.imshow(load_np_image(data[cosine_similarity_id]))
    plt.title(data[cosine_similarity_id]['title'], wrap=True)
    plt.yticks([])
    plt.xticks([])
    plt.subplot(4, 2, 8)
    plt.imshow(np.load(f'depth_maps/{cosine_similarity_id}.npy'))
    plt.title(f'Cosine similarity: {cosine_similarity:.3f}')
    plt.yticks([])
    plt.xticks([])
    plt.show()      

    bar.close() 

for id_ in data:
    if data[id_]['artistName'] is not None and 'vermeer' in data[id_]['artistName'].lower():
        plot_distances(id_)
        break

In [ ]:
#hierarchical clustering

from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import dendrogram, linkage, ward, average, single, complete


distance_matrix = np.load('eucled_distance_matrix.npy')
## for SSIM
# distance_matrix = 1 - distance_matrix
# np.fill_diagonal(distance_matrix, 0)


condensed = squareform(distance_matrix)

linked = ward(condensed)



In [ ]:
#dendorgam

plt.figure(figsize=(10, 7))
dendrogram(linked, truncate_mode='lastp', p=50, color_threshold=9.5)  # Only show the last 50 merged clusters
plt.axhline(y=9.5, color='r', linestyle='--')
plt.ylabel('Razdalja')
plt.xlabel('Gruče (število slik)')
plt.show()

In [ ]:
# Clusers

from scipy.cluster.hierarchy import fcluster

max_clusters = 4

# Cut the dendrogram and get the cluster labels
cluster_labels = fcluster(linked, t=max_clusters, criterion='maxclust')
print(np.unique(cluster_labels, return_counts=True))

In [ ]:
# data for raw clustering

depth_map_matrix = np.zeros((len(data), 10000))
for i, id_ in enumerate(data):
    depth_map = np.load(f'depth_maps_100/{id_}.npy')
    depth_map_matrix[i] = depth_map.flatten()

In [ ]:
# kmeans

kmeans = KMeans(n_clusters=5, random_state=42)
cluster_labels = kmeans.fit_predict(depth_map_matrix)

In [ ]:
#  hdbscan
import hdbscan

hdbscan = hdbscan.HDBSCAN(min_cluster_size=10, gen_min_span_tree=True)
cluster_labels = hdbscan.fit_predict(depth_map_matrix)


In [ ]:
#T-SNE
from sklearn.manifold import TSNE

#load distance-matrix
distance_matrix = np.load('ssim_distance_matrix.npy')
distance_matrix = 1 - distance_matrix
np.fill_diagonal(distance_matrix, 0)

#T-SNE
tsne = TSNE(n_components=2,
            metric='precomputed', 
            init='random', 
            random_state=1, )
            ## exta parameters
            # perplexity=50,
            # early_exaggeration=30,
            # learning_rate=200,
            # n_iter=3000,
            # n_iter_without_progress=300,
            # min_grad_norm=1e-07,
            # method='barnes_hut')
embedded_space = tsne.fit_transform(distance_matrix)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(embedded_space[:, 0], embedded_space[:, 1], c=cluster_labels, alpha=0.6)  # Plotting all points
plt.legend(*scatter.legend_elements(), title="Gruče")
plt.xlabel('t-SNE dimenzija 1')
plt.ylabel('t-SNE dimenzija 2')
plt.show()

In [ ]:
#MDS
from sklearn.manifold import MDS

#load distance-matrix
distance_matrix = np.load('distance_matrix.npy')

#MDS
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
embedded_space = mds.fit_transform(distance_matrix)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(embedded_space[:, 0], embedded_space[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6)  # Plotting all points
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title('t-SNE Graph of the Distance Matrix')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()

In [ ]:
# Save distance matrix as CSV

distance_matrix = np.load('eucled_distance_matrix.npy')

#save as csv
np.savetxt('distance_matrix.csv', distance_matrix, delimiter=',')
    

In [ ]:
# cluster explanation with silhouette score
from sklearn.metrics import silhouette_samples

silhouette_vals = silhouette_samples(depth_map_matrix, cluster_labels)


In [ ]:
# Identify an example with the highest silhouette score in each cluster

unique_clusters = np.unique(cluster_labels)
top_examples = {}

for cluster in unique_clusters:
    cluster_indices = np.where(cluster_labels == cluster)[0]

    # Silhouette scores of samples in the current cluster
    cluster_silhouette_vals = silhouette_vals[cluster_indices]
   
    top_indices = cluster_indices[np.argsort(-cluster_silhouette_vals)[:5]]
    top_examples[cluster] = [(index, silhouette_vals[index]) for index in top_indices]

In [ ]:
# Plot clusters with central examples

keys = list(data.keys())
fig = plt.figure(figsize=(6,9))
i = 1
first = True
for ci, cluster in enumerate(top_examples.values()):
    for (index, silhouette_val) in cluster:
        painting = data[keys[index]]
        depth_map = depth_map_matrix[index].reshape(100, 100)
        plt.subplot(5, 4, i)
        plt.imshow(depth_map, cmap='viridis')
        if first:
            plt.title(f'Gruča {ci + 1}')
            first = False
        plt.axis('off')
        i += 4
    i = ci + 2
    first = True
plt.tight_layout()
plt.show()

In [ ]:
# Example of some paintings with their depth maps and cluster labels

for i, id_ in enumerate(data):
    if i > 2:
        break
    painting = data[id_]
    print(painting['artistName'], ', ', painting['title'], ', ', painting['completitionYear'])
    plt.subplot(1, 2, 1)
    image = load_np_image(painting)
    plt.imshow(image)
    plt.title(painting['title'])
    plt.axis('off')
    plt.subplot(1, 2, 2)
    depth_map = predict_depth(image)
    plt.imshow(depth_map, cmap='viridis')
    plt.title(f'Gruča: {cluster_labels[i]}')    
    plt.axis('off')
    plt.show()
    

Method comparison

In [ ]:
# COMPARE DEPTH MAPS WITH FACES

import time

def compare_faces_wtih_depth(painting, plot=False):
    f = 17 #mm
    sensor_height = 32 #mm
    face_height = 185 #mm
    faces_rf = ast.literal_eval(painting['retinaface'])
    if len(faces_rf) == 0:
        return -1, -1
    points_x = []
    points_y = []
    points_z = []
    depth_map_z = []
    image = load_np_image(painting)

    for i, face in enumerate(faces_rf):  
        if len(face) == 0:
            break
        x1, y1, x2, y2 = faces_rf[face]['facial_area'][0], faces_rf[face]['facial_area'][1], faces_rf[face]['facial_area'][2], faces_rf[face]['facial_area'][3]
        face_height_px = y2 - y1
        image_height = image.shape[0]
        d = (f * face_height * image_height) / (face_height_px * sensor_height)
        points_x.append((x1+x2)/2)
        points_y.append((y1+y2)/2)
        points_z.append(d)
        depth_map_z.append(faces_rf[face]['depth'])

    if len(points_x) == 0:
        return -1, -1

    #points_x = np.array(points_x) / image.shape[1]
    #points_y = np.array(points_y) / image.shape[0]

    points_z = np.max(points_z) - np.array(points_z) # rotate coordinate system
    points_z = (points_z - np.mean(points_z)) / np.std(points_z) # normalize
    depth_map_z = np.array(depth_map_z)
    depth_map_z = (depth_map_z - np.mean(depth_map_z)) / np.std(depth_map_z) # normalize

    if (plot):
        depth_map = predict_depth(image)
        depth_map_norm = (depth_map - np.mean(depth_map)) / np.std(depth_map)
        colormap_name = 'cool'
        cmap = plt.cm.get_cmap(colormap_name)
        copy = image.copy()
        color_z = points_z + np.abs(np.min(points_z))
        color_z = color_z / np.max(color_z)
        
        for i, face in enumerate(faces_rf):
            x1, y1, x2, y2 = faces_rf[face]['facial_area'][0], faces_rf[face]['facial_area'][1], faces_rf[face]['facial_area'][2], faces_rf[face]['facial_area'][3]
            
            color = cmap(color_z[i])
            copy = cv2.rectangle(copy, (x1, y1), (x2, y2), (color[0] *255, color[1] *255, color[2] *255), 2)
            color = cmap(depth_map_z[i])
            copy = cv2.rectangle(copy, (x1-5, y1-5), (x2+5, y2+5), (color[0] *255, color[1] *255, color[2] *255), 2)
            #add text
            #copy = cv2.putText(copy, str(round(points_z[i], 2)), (int((x1+x2)/2), int((y1+y2)/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            #copy = cv2.putText(copy, str(round(depth_map_z[i], 2)), (int((x1+x2)/2), int((y1+y2)/2) + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        plt.imshow(copy)
        plt.xticks([])
        plt.yticks([])
        plt.show()

        points_x = np.array(points_x) / image.shape[1] * depth_map.shape[1]
        points_y = np.array(points_y) / image.shape[0] * depth_map.shape[0]

        x, y = np.meshgrid(np.arange(depth_map.shape[1]), np.arange(depth_map.shape[0]))
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        depth_normalized = (depth_map - np.min(depth_map)) / (np.max(depth_map) - np.min(depth_map))
        ax.plot_surface(y, x, depth_map_norm, rstride=1, cstride=1, facecolors=cm.viridis(depth_normalized), linewidth=0, antialiased=False, shade=False, alpha=0.05)
        plot = ax.scatter(points_y, points_x, points_z, c=points_z, s=150, cmap=colormap_name, alpha=1)
        ax.view_init(azim=30, elev=80)
        fig.colorbar(plot)
        plt.show()

    mse = np.sum(np.square(points_z - depth_map_z)) / len(points_z)
    
    return mse


In [ ]:
# Run comparison of depth maps with faces

model_comparison_mse = {}

for id_ in data:
    n_faces = len(ast.literal_eval(data[id_]['retinaface']))
    if n_faces > 1:
        mse = compare_faces_wtih_depth(data[id_])
        model_comparison_mse[id_] = mse

        

In [ ]:
# Sort by MSE

sorted_mse = {k: v for k, v in sorted(model_comparison_mse.items(), key=lambda item: item[1])}

In [ ]:
# Method comparison examples

keys = model_comparison_mse.keys()

dobre_slike = [
    'O Almoço dos Barqueiros',
    'Forgotten People',
    'Camille Monet and a Child in the Artist’s Garden in Argenteuil',
    'Family feast'
]

count = 0
for key in keys:
    n_faces = len(ast.literal_eval(data[key]['retinaface']))
    mse = model_comparison_mse[key]
    # plot on graph where n_faces is x and mse is y
    plt.scatter(n_faces, mse, c='mediumseagreen', alpha=0.2)
    if data[key]['title'] in dobre_slike:
        try:
            print(data[key]['artistName'], data[key]['title'], data[key]['completitionYear'], n_faces, mse)
        except KeyError:
            print(data[key]['artistName'], data[key]['title'], 'None', n_faces, mse)
        compare_faces_wtih_depth(data[key], plot=True)
        if count == 0:
            plt.annotate(data[key]['title'], (n_faces, mse), textcoords="offset points", xytext=(5,2))
        else:
            plt.annotate(data[key]['title'], (n_faces, mse), textcoords="offset points", xytext=(5,0))
        plt.scatter(n_faces, mse, alpha=0.5, s=60, label=data[key]['title'], facecolors='none', edgecolors='black')
        plt.scatter(n_faces, mse, c='mediumseagreen', alpha=0.2)
        count += 1
plt.xlabel('Število obrazov')
plt.ylabel('MSE')
plt.show()


Extra

In [ ]:
#Voxelize


def voxelize_square(depth_map, resolution):
    image_range = np.max(depth_map) - np.min(depth_map)
    depth_norm = (((depth_map - np.min(depth_map)) / image_range) * resolution).astype(np.uint8)
    depth_norm = cv2.resize(depth_norm, (resolution, resolution), interpolation=cv2.INTER_NEAREST)
    points_x = []
    points_y = []
    points_z = []
    for d in range(np.max(depth_norm), np.min(depth_norm), -1):
        indices = np.where(depth_norm >= d)
        points_x.append(indices[0])
        points_y.append(indices[1])
        points_z.append(depth_norm[indices])
    points_x = np.concatenate(points_x)
    points_y = np.concatenate(points_y)
    points_z = np.concatenate(points_z)
    A = np.column_stack((points_x, points_y, np.ones_like(points_x)))
    coefficients, residuals, _, _ = np.linalg.lstsq(A, points_z, rcond=None)
    a, b, c = coefficients
    normal = np.array([a, b, -1]) / np.linalg.norm(np.array([a, b, -1]))
    normal_z0 = np.array([0, 0, 1])
    angle = np.arccos(np.abs(np.dot(normal, normal_z0)))
    
    return angle, np.degrees(angle)

def voxelize_square_no_depth(depth_map, resolution):
    image_range = np.max(depth_map) - np.min(depth_map)
    depth_norm = (((depth_map - np.min(depth_map)) / image_range) * resolution).astype(np.uint8)
    depth_norm = cv2.resize(depth_norm, (resolution, resolution), interpolation=cv2.INTER_NEAREST)
    points_x = []
    points_y = []
    points_z = []
    for d in range(np.max(depth_norm), np.min(depth_norm), -1):
        indices = np.where(depth_norm == d)
        points_x.append(indices[0])
        points_y.append(indices[1])
        points_z.append(depth_norm[indices])
    points_x = np.concatenate(points_x)
    points_y = np.concatenate(points_y)
    points_z = np.concatenate(points_z)
    A = np.column_stack((points_x, points_y, np.ones_like(points_x)))
    coefficients, residuals, _, _ = np.linalg.lstsq(A, points_z, rcond=None)
    a, b, c = coefficients
    normal = np.array([a, b, -1]) / np.linalg.norm(np.array([a, b, -1]))
    normal_z0 = np.array([0, 0, 1])
    angle = np.arccos(np.abs(np.dot(normal, normal_z0)))
    
    return angle, np.degrees(angle)


def voxelize_whole_ratio(depth_map, height, width, resolution=100):
    shrink_ratio = resolution / np.min(depth_map.shape)
    depth_ratio = resolution / np.min([width, height])
    depth_norm = cv2.resize(depth_map, (0, 0), fx=shrink_ratio, fy=shrink_ratio, interpolation=cv2.INTER_NEAREST)
    depth_norm = ((depth_norm - np.min(depth_norm)) * depth_ratio).astype(np.uint64) 
    points_x = []
    points_y = []
    points_z = []
    for d in range(np.max(depth_norm), np.min(depth_norm), -1):
        indices = np.where(depth_norm >= d)
        points_x.append(indices[0])
        points_y.append(indices[1])
        points_z.append(depth_norm[indices])
    points_x = np.concatenate(points_x)
    points_y = np.concatenate(points_y)
    points_z = np.concatenate(points_z)
    A = np.column_stack((points_x, points_y, np.ones_like(points_x)))
    coefficients, residuals, _, _ = np.linalg.lstsq(A, points_z, rcond=None)
    a, b, c = coefficients
    normal = np.array([a, b, -1]) / np.linalg.norm(np.array([a, b, -1]))
    normal_z0 = np.array([0, 0, 1])
    angle = np.arccos(np.abs(np.dot(normal, normal_z0)))
    
    return angle, np.degrees(angle)

def voxelize_whole_ratio_no_depth(depth_map, height, width, resolution=100):
    shrink_ratio = resolution / np.min(depth_map.shape)
    depth_ratio = resolution / np.min([width, height])
    depth_norm = cv2.resize(depth_map, (0, 0), fx=shrink_ratio, fy=shrink_ratio, interpolation=cv2.INTER_NEAREST)
    depth_norm = ((depth_norm - np.min(depth_norm)) * depth_ratio).astype(np.uint64) 
    points_x = []
    points_y = []
    points_z = []
    for d in range(np.max(depth_norm), np.min(depth_norm), -1):
        indices = np.where(depth_norm == d)
        points_x.append(indices[0])
        points_y.append(indices[1])
        points_z.append(depth_norm[indices])
    points_x = np.concatenate(points_x)
    points_y = np.concatenate(points_y)
    points_z = np.concatenate(points_z)
    A = np.column_stack((points_x, points_y, np.ones_like(points_x)))
    coefficients, residuals, _, _ = np.linalg.lstsq(A, points_z, rcond=None)
    a, b, c = coefficients
    normal = np.array([a, b, -1]) / np.linalg.norm(np.array([a, b, -1]))
    normal_z0 = np.array([0, 0, 1])
    angle = np.arccos(np.abs(np.dot(normal, normal_z0)))
    
    return angle, np.degrees(angle)

def voxelize_no_back(depth_map, height, width, resolution=100):
    shrink_ratio = resolution / np.min(depth_map.shape)
    depth_ratio = resolution / np.min([width, height])
    depth_norm = cv2.resize(depth_map, (0, 0), fx=shrink_ratio, fy=shrink_ratio, interpolation=cv2.INTER_NEAREST)
    depth_norm = ((depth_norm - np.min(depth_norm)) * depth_ratio).astype(np.uint64) 
    points_x = []
    points_y = []
    points_z = []
    for d in range(int(np.max(depth_norm)), int(np.min(depth_norm)) + 1, -1):
        indices = np.where(depth_norm >= d)
        points_x.append(indices[0])
        points_y.append(indices[1])
        points_z.append(depth_norm[indices])
    points_x = np.concatenate(points_x)
    points_y = np.concatenate(points_y)
    points_z = np.concatenate(points_z)
    A = np.column_stack((points_x, points_y, np.ones_like(points_x)))
    coefficients, residuals, _, _ = np.linalg.lstsq(A, points_z, rcond=None)
    a, b, c = coefficients
    normal = np.array([a, b, -1]) / np.linalg.norm(np.array([a, b, -1]))
    normal_z0 = np.array([0, 0, 1])
    angle = np.arccos(np.abs(np.dot(normal, normal_z0)))
    
    return angle, np.degrees(angle)

def voxelize_no_back_no_depth(depth_map, height, width, resolution=100):
    shrink_ratio = resolution / np.min(depth_map.shape)
    depth_ratio = resolution / np.min([width, height])
    depth_norm = cv2.resize(depth_map, (0, 0), fx=shrink_ratio, fy=shrink_ratio, interpolation=cv2.INTER_NEAREST)
    depth_norm = ((depth_norm - np.min(depth_norm)) * depth_ratio).astype(np.uint64) 
    points_x = []
    points_y = []
    points_z = []
    for d in range(int(np.max(depth_norm)), int(np.min(depth_norm)) + 1, -1):
        indices = np.where(depth_norm == d)
        points_x.append(indices[0])
        points_y.append(indices[1])
        points_z.append(depth_norm[indices])
    points_x = np.concatenate(points_x)
    points_y = np.concatenate(points_y)
    points_z = np.concatenate(points_z)
    A = np.column_stack((points_x, points_y, np.ones_like(points_x)))
    coefficients, residuals, _, _ = np.linalg.lstsq(A, points_z, rcond=None)
    a, b, c = coefficients
    normal = np.array([a, b, -1]) / np.linalg.norm(np.array([a, b, -1]))
    normal_z0 = np.array([0, 0, 1])
    angle = np.arccos(np.abs(np.dot(normal, normal_z0)))
    
    return angle, np.degrees(angle)


In [ ]:
# visualize results

def show(painting):
    image = load_np_image(painting)
    depth_map = predict_depth(image)
    print(painting['artistName'], ' - ', painting['title'], ' - ',painting['completitionYear'])
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1, 2, 2)
    plt.imshow(depth_map)
    plt.xticks([])
    plt.yticks([])
    plt.show()

with open('image_data.json') as json_file:
    data = json.load(json_file)

    count = 0
    for id_ in data:
        year = data[id_]['completitionYear']
        if year == None:
            continue

        if year > 1700 and year < 1800:
            show(data[id_])

       